In [186]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
# conda install pytorch torchvision -c pytorch 
import tensorflow as tf
import torch
from torch.utils.data import TensorDataset # 텐서데이터셋
from torch.utils.data import DataLoader # 데이터로더
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import LeavePOut


In [187]:
# data/output_df의 파일들과 data/weather의 파일들을 읽어온다.
seoul_output_df = pd.read_csv('data/output_df/seoul_spring_df.csv')
seoul_spring_df = pd.read_csv('data/weather/seoul_spring.csv')

# 위에 파일들을 합친다.
seoul_df = pd.merge(seoul_output_df, seoul_spring_df, on='일시', how='left')

# 출력
seoul_df



,일시,일반봄배추:면적 (ha),생산량 (톤),평균기온(°C),최고기온(°C),최저기온(°C),월합강수량(00~24h만)(mm),합계 일사량(MJ/m2)
0,2000,72,2660,6.3,18.9,-4.8,3.1,389.50
1,2000,72,2660,11.9,23.5,1.8,30.7,440.90
2,2000,72,2660,17.5,30.5,7.9,75.2,443.69
3,2001,8,353,5.0,21.0,-6.1,18.1,396.53
4,2001,8,353,13.6,28.2,2.1,12.3,492.04
...,...,...,...,...,...,...,...,...
61,2020,0,1,11.1,23.8,1.9,16.9,637.52
62,2020,0,1,18.0,30.0,9.2,112.4,564.83
63,2021,5,228,9.0,22.9,-1.2,110.9,476.94
64,2021,5,228,14.2,28.2,3.1,124.1,558.28


In [188]:
# seoul_df 컬럼 = "일시",	"일반봄배추:면적 (ha)",	"생산량 (톤)",	"평균기온(°C)" 만 남기기
seoul_df = seoul_df[["일시","일반봄배추:면적 (ha)",	"생산량 (톤)",	"평균기온(°C)"]]
seoul_df


,일시,일반봄배추:면적 (ha),생산량 (톤),평균기온(°C)
0,2000,72,2660,6.3
1,2000,72,2660,11.9
2,2000,72,2660,17.5
3,2001,8,353,5.0
4,2001,8,353,13.6
...,...,...,...,...
61,2020,0,1,11.1
62,2020,0,1,18.0
63,2021,5,228,9.0
64,2021,5,228,14.2


In [189]:
# 입력데이터 정규화 (StandardScaler)
scaler = StandardScaler()
scaler.fit(seoul_df[["일반봄배추:면적 (ha)",	"생산량 (톤)",	"평균기온(°C)"]])
seoul_df[["일반봄배추:면적 (ha)",	"생산량 (톤)",	"평균기온(°C)"]] = scaler.transform(seoul_df[["일반봄배추:면적 (ha)",	"생산량 (톤)",	"평균기온(°C)"]])
seoul_df





,일시,일반봄배추:면적 (ha),생산량 (톤),평균기온(°C)
0,2000,1.754873,2.410005,-1.198366
1,2000,1.754873,2.410005,-0.102853
2,2000,1.754873,2.410005,0.992660
3,2001,-0.435607,-0.304781,-1.452681
4,2001,-0.435607,-0.304781,0.229714
...,...,...,...,...
61,2020,-0.709417,-0.719000,-0.259354
62,2020,-0.709417,-0.719000,1.090474
63,2021,-0.538286,-0.451876,-0.670172
64,2021,-0.538286,-0.451876,0.347090


In [190]:
# sequence to sequence
def split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(sequence)-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix, :-1], sequence[end_ix, -1]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

# 데이터셋 생성
n_steps = 7
X, y = split_sequence(seoul_df.values, n_steps)

# 데이터셋 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# 데이터셋 변환
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2])
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2])

In [191]:
# 모델 생성
model = Sequential()
model.add(LSTM(128, activation='relu', input_shape=(n_steps, X.shape[2])))
model.add(Dense(1))
model.summary()

# 모델 컴파일
model.compile(optimizer='adam', loss='mse')

# 모델 학습
model.fit(X_train, y_train, epochs=100, verbose=0)


Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_20 (LSTM)              (None, 128)               67584     
                                                                 
 dense_10 (Dense)            (None, 1)                 129       
                                                                 
Total params: 67,713
Trainable params: 67,713
Non-trainable params: 0
_________________________________________________________________


2022-12-19 15:20:13.736493: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


In [192]:
# 모델 평가 
y_pred = model.predict(X_test)
print('MSE :', mean_squared_error(y_test, y_pred))
print('MAE :', mean_absolute_error(y_test, y_pred))
print('R2 :', r2_score(y_test, y_pred))

# 모델 저장
model.save('model/model.h5')



1/1 [==============================] - 0s 128ms/step
MSE : 2.1451494210267925
MAE : 1.3743905688696822
R2 : -0.4437878120624863


2022-12-19 15:20:28.122262: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


In [193]:
# 모델 예측
y_pred = model.predict(X_test)

# 예측값 역정규화
y_pred = scaler.inverse_transform(y_pred)

# 실제값 역정규화
y_test = scaler.inverse_transform(y_test.reshape(-1, 1))

# 예측값 출력
y_pred

1/1 [==============================] - 0s 25ms/step


ValueError: non-broadcastable output operand with shape (12,1) doesn't match the broadcast shape (12,3)